# Reading a Neon dataset/recording
In this tutorial, we will show how to load a single Neon recording downloaded from [Pupil Cloud](https://docs.pupil-labs.com/neon/pupil-cloud/).

## Reading sample data
We will use a sample recording produced by the NCC Lab called `OfficeWalk`. It's a project with 2 recordings and multiple enrichments and can be downloaded with the `get_sample_data()` function:

In [1]:
import sys
import numpy as np
from scipy.ndimage import gaussian_filter
from pyneon import get_sample_data, NeonDataset, NeonRecording

sample_dir = get_sample_data("OfficeWalk")

The `OfficeWalk` data has the following structure:

```plaintext
OfficeWalk
├── Timeseries Data
│   ├── walk1-e116e606
│   │   ├── info.json
│   │   ├── gaze.csv
│   │   └── ....
│   ├── walk2-93b8c234
│   │   ├── info.json
│   │   ├── gaze.csv
│   │   └── ....
|   ├── enrichment_info.txt
|   └── sections.csv
├── OfficeWalk_FACE-MAPPER_FaceMap
├── OfficeWalk_MARKER-MAPPER_TagMap_csv
└── OfficeWalk_STATIC-IMAGE-MAPPER_ManualMap_csv
```

The `Timeseries Data` folder contains what PyNeon calls a `NeonDataset`. It contains multiple recordings, each with its own `info.json` file and data files. These recordings can either be loaded individually as `NeonRecording`s or as a wholist `NeonDataset`.

If loading a `NeonDataset`, specify the path to the `Timeseries Data` folder to create a `NeonDataset` object:

In [ ]:
dataset_dir = sample_dir / "Timeseries Data"
dataset = NeonDataset(dataset_dir)
print(dataset)

NeonDataset has a `recordings` attribute that contains a list of `NeonRecording` objects. These `NeonRecording` objects can be accessed by their index.

In [ ]:
first_recording = dataset[0]
print(type(first_recording))

Alternatively, one can directly load a single `NeonRecording` by specifying the path to the recording's folder:

In [ ]:
recording_dir = dataset_dir / "walk1-e116e606"
recording = NeonRecording(recording_dir)
print(type(recording))

## Data and metadata of a NeonRecording
An overview of basic metadata and contents of a `NeonRecording` can be obtained by printing the object. An initiated `NeonRecording` locates data files in the recording directory but does not load them until requested to be memory efficient.

In [ ]:
print(recording)

Individual data streams can be accessed as properties of the `NeonRecording` object. For example, the gaze data can be accessed as `recording.gaze`, and upon accessing, the tabular data is loaded into memory.

In [ ]:
print(f"recording._gaze size before accessing `gaze`: {sys.getsizeof(recording._gaze)}")

gaze = recording.gaze
print(f"recording.gaze is of type: {type(gaze)}")
print(f"recording._gaze size after accessing `gaze`: {sys.getsizeof(recording._gaze)}")

We can access the timeseries data in the gaze stream as a pandas DataFrame by accessing the `data` attribute of the gaze stream. The columns of the DataFrame include `timestamp [ns]` and channel data columns. During loading, PyNeon strips the redundant `section id` and `recording id` columns and adds a more human-readable `time [s]` column to represent the time of each sample in seconds relative to the start of the data stream.

In [ ]:
print(gaze.data.head())

PyNeon also automatically sets the column datatype to appropriate types, such as `Int64` for timestamps, `Int32` for event IDs, and `float64` for float data.

In [ ]:
print(gaze.data.dtypes)

## Data streams and events

Up to this point, PyNeon simply reads and re-organizes the raw .csv files. Let's plot some samples from the `gaze` and `eye_states` streams and a saccade from the `saccades` events.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

gaze_color = "royalblue"
gyro_color = "darkorange"

imu = recording.imu
saccades = recording.saccades

# Create a figure
fig, ax = plt.subplots(figsize=(10, 5))
ax2 = ax.twinx()
ax.yaxis.label.set_color(gaze_color)
ax2.yaxis.label.set_color(gyro_color)

# Visualize the 2nd saccade
saccade = saccades.data.iloc[1]
ax.axvspan(
    saccade["start timestamp [ns]"], saccade["end timestamp [ns]"], color="lightgray"
)
ax.text(
    (saccade["start timestamp [ns]"] + saccade["end timestamp [ns]"]) / 2,
    1050,
    "Saccade",
    horizontalalignment="center",
)

# Visualize gaze x and pupil diameter left
sns.scatterplot(
    ax=ax,
    data=gaze.data.head(100),
    x="timestamp [ns]",
    y="gaze x [px]",
    color=gaze_color,
)
sns.scatterplot(
    ax=ax2,
    data=imu.data.head(60),
    x="timestamp [ns]",
    y="gyro x [deg/s]",
    color=gyro_color,
)

It's apparent that at the beginning of the recording, there are some missing data points in both the `gaze` and `imu` streams. This is presumably due to the time it takes for the sensors to start up and stabilize. We will show how to handle missing data using resampling in the next tutorial. For now, it's important to be aware of these gaps and that it will require great caution to assume the data is continuously and equally sampled.

PyNeon also calculates the effective (as opposed to the nominal) sampling frequency of each stream by dividing the number of samples by the duration of the recording.

In [ ]:
print(
    f"Gaze: nominal sampling frequency = {gaze.sampling_freq_nominal}, "
    f"effective sampling frequency = {gaze.sampling_freq_effective}"
)
print(
    f"IMU: nominal sampling frequency = {recording.imu.sampling_freq_nominal}, "
    f"effective sampling frequency = {recording.imu.sampling_freq_effective}"
)

## Visualizing gaze heatmap
Finally, we will show how to plot a heatmap of the gaze/fixation data. We will further add violin plots to show the distribution in x and y

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec
from scipy.ndimage import gaussian_filter

# Generate the heatmap data
x_edges = np.linspace(0, 1600, 160)
y_edges = np.linspace(0, 1200, 120)

# Sample gaze and fixation data (replace with your actual data)
gaze_x = gaze.data["gaze x [px]"]
gaze_y = gaze.data["gaze y [px]"]
gaze_heatmap, _, _ = np.histogram2d(gaze_x, gaze_y, bins=(x_edges, y_edges))
gaze_heatmap = gaussian_filter(gaze_heatmap, sigma=3)

fixations = recording.fixations
fix_x = fixations.data["fixation x [px]"]
fix_y = fixations.data["fixation y [px]"]

# Create a figure with gridspec layout
fig = plt.figure(figsize=(10, 8))
gs = gridspec.GridSpec(
    4, 4, width_ratios=[0.2, 1, 1, 0.2], height_ratios=[0.2, 1, 1, 0.2]
)

# Main heatmap plot
ax_main = fig.add_subplot(gs[1:3, 1:3])
ax_main.imshow(
    gaze_heatmap.T, cmap="inferno", extent=[0, 1600, 0, 1200], origin="lower"
)
ax_main.scatter(fix_x, fix_y, color="white", s=10, alpha=0.3)
ax_main.set_aspect("equal", "box")
ax_main.set_xlabel("Scene camera x [px]")
ax_main.set_ylabel("Scene camera y [px]")

# Violin plot for x distribution (top)
ax_top = fig.add_subplot(gs[0, 1:3], sharex=ax_main)
ax_top.violinplot(fix_x, vert=False, showmedians=True)
ax_top.axis("off")  # Hide x-axis labels and ticks

# Violin plot for y distribution (right)
ax_right = fig.add_subplot(gs[1:3, 3], sharey=ax_main)
ax_right.violinplot(fix_y, vert=True, showmedians=True)
ax_right.axis("off")  # Hide y-axis labels and ticks

# Adjust layout
plt.subplots_adjust(wspace=0.05, hspace=0.05)

plt.show()

we can neatly see that the recorded data shows a centre-bias, which is a well-known effect from eye statistics. In y, we can see that fixations tend to occur below the horizon, which is indicative of a walking task where a participant looks at the floor in front of them more often